# CLI commands

> Console commands added by the nbdev library

In [1]:
# default_exp cli

In [2]:
# export
from nbdev.imports import *
from nbdev.export import *
from nbdev.sync import *
from nbdev.export2html import *
from nbdev.test import *
from fastscript.fastscript import call_parse, Param

## Navigating from notebooks to script and back

In [3]:
#export
@call_parse
def nbdev_build_lib(fname:Param("A notebook name or glob to convert", str)=None):
    notebook2script(fname=fname)

In [4]:
#export
@call_parse
def nbdev_update_lib(fname:Param("A notebook name or glob to convert", str)=None):
    script2notebook(fname=fname)

In [5]:
#export
@call_parse
def nbdev_diff_nbs(): diff_nb_script()

## Parallel execution

In [6]:
#export
from multiprocessing import Process, Queue
import concurrent

In [7]:
#export
def num_cpus():
    "Get number of cpus"
    try:                   return len(os.sched_getaffinity(0))
    except AttributeError: return os.cpu_count()

In [8]:
#export 
class ProcessPoolExecutor(concurrent.futures.ProcessPoolExecutor):
    "Like `concurrent.futures.ProcessPoolExecutor` but handles 0 `max_workers`."
    def __init__(self, max_workers=None, on_exc=print, **kwargs):
        self.not_parallel = max_workers==0
        self.on_exc = on_exc
        if self.not_parallel: max_workers=1
        super().__init__(max_workers, **kwargs)

    def map(self, f, items, *args, **kwargs):
        g = partial(f, *args, **kwargs)
        if self.not_parallel: return map(g, items)
        try: return super().map(g, items)
        except Exception as e: self.on_exc(e)

In [9]:
#export 
def parallel(f, items, *args, n_workers=None, **kwargs):
    "Applies `func` in parallel to `items`, using `n_workers`"
    if n_workers is None: n_workers = min(16, num_cpus())
    with ProcessPoolExecutor(n_workers) as ex:
        r = ex.map(f,items, *args, **kwargs)
        return list(r)

In [10]:
import time,random

def add_one(x, a=1): 
    time.sleep(random.random()/100)
    return x+a

inp,exp = range(50),range(1,51)
test_eq(parallel(add_one, inp, n_workers=2), list(exp))
test_eq(parallel(add_one, inp, n_workers=0), list(exp))
test_eq(parallel(add_one, inp, n_workers=1, a=2), list(range(2,52)))
test_eq(parallel(add_one, inp, n_workers=0, a=2), list(range(2,52)))

## Extracting tests

In [11]:
# export
def _test_one(fname, flags=None):
    time.sleep(random.random())
    print(f"testing: {fname}")
    try: test_nb(fname, flags=flags)
    except Exception as e: print(e)

In [12]:
# export
@call_parse
def nbdev_test_nbs(fname:Param("A notebook name or glob to convert", str)=None,
                   flags:Param("Space separated list of flags", str)=None):
    if flags is not None: flags = flags.split(' ')
    if fname is None: 
        files = [f for f in Config().nbs_path.glob('*.ipynb') if not f.name.startswith('_')]
    else: files = glob.glob(fname)
        
    # make sure we are inside the notebook folder of the project
    os.chdir(Config().nbs_path)
    parallel(_test_one, files, flags=flags)

## Building documentation

In [13]:
#export
import time,random,warnings

In [14]:
#export
def _leaf(k,v):
    url = 'external_url' if "http" in v else 'url'
    if url=='url': v=v+'.html'
    return {'title':k, url:v, 'output':'web,pdf'}

In [15]:
#export
_k_names = ['folders', 'folderitems', 'subfolders', 'subfolderitems']
def _side_dict(title, data, level=0):
    k_name = _k_names[level]
    level += 1
    res = [(_side_dict(k, v, level) if isinstance(v,dict) else _leaf(k,v))
        for k,v in data.items()]
    return ({k_name:res} if not title
            else res if title.startswith('empty')
            else {'title': title, 'output':'web', k_name: res})

In [16]:
#export
def make_sidebar():
    "Making sidebar for the doc website"
    if not (Config().doc_path/'sidebar.json').exists():
        warnings.warn("No data for the sidebar available")
        sidebar_d = {}
    else: sidebar_d = json.load(open(Config().doc_path/'sidebar.json', 'r'))
    res = _side_dict('Sidebar', sidebar_d)
    res = {'entries': [res]}
    res_s = yaml.dump(res, default_flow_style=False)
    res_s = res_s.replace('- subfolders:', '  subfolders:').replace(' - - ', '   - ')
    res_s = f"""
#################################################
### THIS FILE WAS AUTOGENERATED! DO NOT EDIT! ###
#################################################
# Instead edit {Config().doc_path/'sidebar.json'}
"""+res_s
    open(Config().doc_path/'_data/sidebars/home_sidebar.yml', 'w').write(res_s)

In [17]:
# export
def convert_one(fname):
    time.sleep(random.random())
    print(f"converting: {fname}")
    try: convert_nb(fname)
    except Exception as e: print(e)

In [18]:
# export
def convert_all(fname=None, force_all=False):
    "Convert all notebooks matching `fname` to html files"
    if fname is None: 
        files = [f for f in Config().nbs_path.glob('*.ipynb') if not f.name.startswith('_')]
    else: files = glob.glob(fname)
    if not force_all:
        # only rebuild modified files
        files,_files = [],files.copy()
        for fname in _files:
            fname_out = Config().doc_path/'.'.join(fname.with_suffix('.html').name.split('_')[1:])
            if not fname_out.exists() or os.path.getmtime(fname) >= os.path.getmtime(fname_out):
                files.append(fname)
    if len(files)==0: print("No notebooks were modified")          
    parallel(convert_one, files)

In [19]:
# export
@call_parse
def nbdev_build_docs(fname:Param("A notebook name or glob to convert", str)=None,
         force_all:Param("Rebuild even notebooks that haven't changed", bool)=False):
    convert_all(fname=fname, force_all=force_all)
    make_sidebar()

## Export -

In [23]:
#hide
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 05_cli.ipynb.
Converted 09_index.ipynb.
